## Parametri e iperparametri 
Sappiamo che esiste il metodo di suddivisione dei dati a disposizione, detto **hold out**, in *training set* (tipicamente il 70%), serve per trovare gli iperparametri e *validation set* (30%), il quale serve per fare *tuning* degli *iperparametri*. Ad esempio serve per trovare
- Il *learning rate* adatto
- Il *grado del polinomio* che meglio modella il dominio
Si chiamano cosi poiché non è il modello di regressione che li definisce, dobbiamo stabilirli noi, ma cion nonostante, servono a predire correttamente.
### Come trovare i migliori iperparametri
Fondamentalmente questa ricerca è del tutto empirica. Si fanno più esperimenti facendo variare un iperparametro alla volta, controllando qual'è quello che fitta meglio. Pertanto in qualche modo, il modello viene ritagliato anche sul validation set (anche se indirettamente, poiché controllato da noi: il modello non si modifica autonomamente sul validation set), poiché è su quello che si fa *fine tuning* del modello. Pertanto questo aspetto non va sottovalutato, in quanto cerchiamo di minimizzare l'errore, osservando come si comporta su nuovi dati. A questo proposito, non è sufficiente suddividere il dataset in due parti, per questo motivo si suddivide ulteriormente in 
- training set: per trainare il modello
- validation set: usato per trovare gli iperparametri migliori
- test set: il modello trovato, che sembra il più opportuno sulla base degli iperparametri trovati, va testato per verificare che il modello dia risultati quanto meno compatibili con quelli del validation set.
È ovvio che se anche sul test set ci mettiamo a correggere gli iperparametri, la procedura di suddivisione in tre parti non serve a nulla.

#### Cosa fare se il test set non va
Qualora il test set non desse i risultati sperati, anche modificando gli iperparametri sulla base di quanto visto con il validation set, la cosa migliore da fare è andare a modificare il modello magari rivedendo le feature: togliendone alcune, o aggiungendone altre.

### K-Fold Cross Validation
Il dataset viene suddiviso in $k$ sottoinsiemi disgiunti. Un sottoinsieme viene usato come validation set, mentre gli altri $k-1$ come training set.
Questo approccio verrà ripetuto permutando tutti i set, cioè praticamente si esegue $k$ volte, ogni volta scegliendo un fold-set differente come validation.
In altre parole se avessimo 6 set 

| Fold 1 | Fold 2 | ... | Fold 6 |
|:------:|:------:|:---:|:------:|

- all'iterazione (1) si fissa *fold 1* come validation set, si usano (*fold 1*, $\cdots$, *fold 6*) come set per trainare il modello.
- all'iterazione (2) si fissa *fold 2* come validation e (*fold 1, fold 3, $\cdots$, *fold 6*) come training set.
- ...

E si prosegue cosi fino a rendere il *Fold 6* validation. In questo modo si producono tanti modelli quanti sono i fold creati, in questo caso 6. Le performance finali saranno date dalla media delle performance di ciascuno dei $k$ modelli.
<img src="imgs/kfoldcross.png" alt="K-Fold Cross Validation" width=500>

#### K-Fold Cross Validation stratificata
Si parla di *K-Fold Cross Validation stratificata* quando imponiamo che tutti i fold siano rappresentati con le stesse caratteristiche. Cioè se ad esempio avessi un modello che valuta l'affidabilità di persone a cui affiare credito (utile alle banche), con *K-Fold Cross Validation stratificata* sarò sicuro che ogni fold di persone avrà la stessa distribuzione di utenti, cioè lo stesso numero di donne, bambini e uomini, ad esempio, stesso numero di nuclei famigliari, ecc.

### Regolarizzazione
Uno dei problemi che insorgono all'uamentare il grado del polinomio, è che i coefficienti $\theta_i$ crescono enormemente. Questo è un problema poiché più abbiamo a che fare con grandi valori e maggiore sarà l'imprecisione del modello.
A tal proposito esiste questo metodo detto di **regolarizzazione** che serve proprio a mantenere bassi i parametri da apprendere, indipendentemente dalla complessità del modello. Il bello di questa tecnica è che è applicabile a qualsiasi modello, anche non di regressione (ad esempio nelle reti neurali).
#### Ridge regression
Per quanto riguarda il caso di regressione lineare, esiste un modello, detto **ridge regression** che incorpora in se la regolarizzazione.
In termini ipotetici, con grado del polinomio pari a $+\infty$ potremmo modellare qualsiasi fenomeno perfettamente poiché ogni punto in aggiunta sarà raggiunto perfettamente dalla curva, tuttavia per gradi già prossimi al 50, i $\theta$ raggiungono valori come
$$
    \theta = -3.88\times 10^6
$$
e ciò provoca forti oscillazioni nella regressione peggiorandone l'accuratezza.
Quello che fa *ridge regression* è cercare di minimizzare, oltre alla funzione d'errore, anche il prodotto
$$
        \lambda \cdot \theta
$$
con $\lambda$ *iperparametro* tale che $0 \leq \lambda < \infty$. Ovviamente se $\lambda = 0$ non avviene alcuna regolarizzazione. Ineffetti questo iperparametro serve proprio a stabilire quanto debba incidere sul modello tale regolarizzazione, se decidiamo che $\lambda = 0$ vuol dire che non ci interessa regolarizzare, viceversa, maggiore sarà questo valore, e maggiore risulterà il peso che ha sul modello.
In altre parole si cerca di fare quanto segue
$$
    \underset{\theta}{\text{minimize}} \sum_{i=1}^m (\theta^T x^{(i)} - y^{(i)})^2 + \lambda\|\theta\|^2_2
$$
con 
$$
    \lambda\|\theta\|^2_2 = \sum_{i=0}^{n-1}\theta_i^2
$$
con $n =$ numero parametri $\theta$ (in altre parole il grado del polinomio).
In altre parole quello che accade è che alla classica funzione errore da minimizzare ($\sum_{i=1}^m (\theta^T x^{(i)} - y^{(i)})^2$), si somma lambda per la **norma al quadrato di $\theta$**, ovvero la sommatoria dell'errore quadratico medio di ogni parametro $\theta_i$, cioè $\sum_{i=0}^{n-1}\theta_i^2$.
Dunque in questo modo forziamo il modello a trovare i theta che minimizzano l'errore, ma che siano anche i più prossimi a 0. In altre parole si riduce a un problema di minimo vincolato (programmazione lineare di ricerca operativa), in cui però il vincolo entra a far parte della funzione obbiettivo.

Come al solito per minimizzare una funzione ne calcoliamo il gradiente (derivando), che nel nostro caso diventa

$$
    \nabla_{\theta} \left(\sum_{i=1}^m (\theta^T x^{(i)} - y^{(i)})^2 + \lambda\|\theta\|^2_2\right) = 2X^T (X\theta - y) + 2\lambda\theta
$$

da cui, eguagliando a zero otteniamo

$$
2X^TX \theta + 2\lambda \theta = 2X^Ty \Rightarrow \theta = (X^T X + \lambda I)^{-1}X^Ty
$$

dove come sempre $X$ è la matrice dei coefficienti noti, e $y$ è il vettore dei valori da predire.

Graficamente otterremmo che, senza regolarizzazione, per un polinomio di grado 50, la curva sarà la seguente
<img src="imgs/noreg.PNG" alt="Grado 50 senza regolarizzazione" width=400>
mentre con regolarizzazione $\lambda = 1$ 
<img src="imgs/reg.PNG" alt="Grado 50 con regolarizzazione" width=400>

#### Come individuare il $\lambda$ migliore
Il metodo più semplice è usare k-fold cross validation. In python $\lambda$ è un iperparametro chiamato $\alpha$ (apha).

```python
from sklearn.linear_model import RidgeCV
...
alphas = 10**np.linspace(-15, 9, 25)
reg_cv = RidgeCV(alphas, cv=5) #5-cross fold
reg_cv.fit(poly.fit_transform(X_train), y_train) #aumento il numero di parametri (il grado del modello) con poly.fit_transform
reg_cv.score(poly.transform(X_val), y_val)
0.01102
reg_cv.alpha_
0.1
```

### Nested cross validation
Determinare gli iperparametri migliori con gli stessi dati usati per trainare il modello, genera score ottimisti. Per migliorare è possibile usare il **Nested Cross Validation**.
Dalla parola *nested* capiamo che sono due cross validation innestate.
<img src="imgs/nestedkcross.PNG" alt="nested kcross" width=400>
Come mostra il grafico sopra, infatti, abbiamo una cross validation esterna (in blu i test fold) che viene suddivisa nuovamente in una cross fold interna (in arancione i test fold). Nell'immagine avremo 5x2 fold. Questo perché per ogni fold esterno avvengono i due interni. Perciò all'$i$-esima iterazione avremo il training fold esterno (escluso dunque il test fold), che verrà a sua volta usato come dataset di partenza per un cross validation interno a 2 fold (training e validtion).
In questo senso
- La cross fold *esterna* serve per valutare l'accuratezza media del modello sui dati
- La cross fold *interna* serve per trovare i migliori iper parametri. 

Lo pseudocodice è il seguente
<img src="imgs/pseudocodicenestedcross.PNG" alt="Pseudocodice" width=550>
Al termine dell'esecuzione avremo, in questo caso 10 modelli. Per trovare il migliore addestreremo su tutti i dati (senza folding) utilizzando per ogni iperparametro quello che ha dato risultato medio migliore. Ad esempio nel nostro caso, avendo come iperparametri solo il grado del polinomio e il peso della regolarizzazione, potrebbe venire fuori che $\lambda = 0.1$, grado del polinomio $= 10$.

### Collinearità
